<a href="https://colab.research.google.com/github/Satwikram/NLP-Implementations/blob/main/Generative%20Models/Llama%202%20Hugging%20Face%20Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Author: Satwik Ram

### Setup

In [1]:
!pip install transformers accelerate

### Setup

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


### Imports

In [3]:
from transformers import AutoTokenizer
import transformers
import torch
# from definations import schema

### Initializing the tokenizer

In [4]:
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

In [5]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
schema = """CREATE TABLE querytool_2023.fhr_vitals (
mother_person_id int4 NULL,
mother_encounter_id int4 NULL,
vitals_event varchar(255) NULL,
normal_low numeric NULL,
normal_high numeric NULL,
result_value varchar(255) NULL,
result_units varchar(255) NULL,
valid_from_date_time timestamp NULL,
id serial4 NOT NULL
);"""

In [7]:
nlq = "What is Laterality"

In [19]:
prompt = f"""[INST] <<SYS>>

    Given the following maternal fetal health database Schema, and Natural Language Question (NLQ) below, translate the NLQ to equivalent SQLlite query. For string matching use LIKE. Do not put the word SQL prior to the answer. Return just the SQL query.

    Schema: {schema}

<</SYS>>

NLQ: {nlq} [/INST]"""


In [20]:
sequences = pipeline(
    prompt,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_length=512,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Result: [INST] <<SYS>>

    Given the following maternal fetal health database Schema, and Natural Language Question (NLQ) below, translate the NLQ to equivalent SQLlite query. For string matching use LIKE. Do not put the word SQL prior to the answer. Return just the SQL query.

    Schema: CREATE TABLE querytool_2023.fhr_vitals (
mother_person_id int4 NULL,
mother_encounter_id int4 NULL,
vitals_event varchar(255) NULL,
normal_low numeric NULL,
normal_high numeric NULL,
result_value varchar(255) NULL,
result_units varchar(255) NULL,
valid_from_date_time timestamp NULL,
id serial4 NOT NULL
);

<</SYS>>

NLQ: What is Laterality [/INST]  SELECT * FROM querytool_2023.fhr_vitals WHERE vitals_event LIKE '%Laterality%';


In [22]:
res = sequences[0]["generated_text"].strip()

In [23]:
res

"[INST] <<SYS>>\n\n    Given the following maternal fetal health database Schema, and Natural Language Question (NLQ) below, translate the NLQ to equivalent SQLlite query. For string matching use LIKE. Do not put the word SQL prior to the answer. Return just the SQL query.\n\n    Schema: CREATE TABLE querytool_2023.fhr_vitals (\nmother_person_id int4 NULL,\nmother_encounter_id int4 NULL,\nvitals_event varchar(255) NULL,\nnormal_low numeric NULL,\nnormal_high numeric NULL,\nresult_value varchar(255) NULL,\nresult_units varchar(255) NULL,\nvalid_from_date_time timestamp NULL,\nid serial4 NOT NULL\n);\n\n<</SYS>>\n\nNLQ: What is Laterality [/INST]  SELECT * FROM querytool_2023.fhr_vitals WHERE vitals_event LIKE '%Laterality%';"

In [24]:
import re

In [32]:
# Define the start and end patterns
start_pattern = r"[INST]"
end_pattern = r"[/INST]"

# Create a regular expression pattern to match everything between start and end
pattern = re.compile(re.escape(start_pattern) + r".*?" + re.escape(end_pattern), re.DOTALL)

# Use re.sub() to replace the matched pattern with an empty string
result_string = re.sub(pattern, "", res)

print(result_string)

  SELECT * FROM querytool_2023.fhr_vitals WHERE vitals_event LIKE '%Laterality%';
